# Taxi Networks simulation

In this notebook, we will demonstrate code that simulates the behavior of taxis
in queueing networks. We are modeling this using a Jackson network, which models
taxis as customers traveling in the network. Taxis queue at each station $$i$$,
and are served by customers arriving at a rate of $$\mu_i$$. After a customer
gets on a taxi, it travels to another station $$j$$ with probability
$$r_{ij}$$. We assume the taxis 

In [26]:
import numpy as np
import scipy.stats as sps
from matplotlib import pyplot as plt
%matplotlib inline

Here we define the a Jackson Network Node. 

In [264]:
class Node:
    '''A Node object in a Jackson network simulates a queue with an exponential
    service time.'''
    def __init__(self, lam, n, r, dt=1):
        '''
        lam    : Service rate (average vehicles per time), constant or a function
                 of number of vehicles in node
        n      : Number of vehicles in the node
        r      : Routing probability from node id to probability
        dt     : Timestep 
        '''
        self.r = r
        self.check_r()
        self.lam = lam if callable(lam) else lambda x: lam
        self.dt = dt
        self.n = n  # Number in queue
        
    def check_r(self):
        assert abs(sum(self.r.values()) - 1.0) < 1e-15, 'Routing probabilities do not sum to 1!'

    def add(self, n):
        self.n += n

    def route_to(self):
        '''Samples a destination from the routing probability distribution'''
        p, s = np.random.sample(), 0
        for id, prob in self.r.items():
            s += prob
            if p < s: return id
        print p, s, id, self.r

    def step(self):
        '''Sample cars from Poisson distribution'''
        num_samples = np.random.poisson(lam=self.lam(self.n) * self.dt)
        destinations = [self.route_to() for _ in range(min(self.n, num_samples))]
        self.n = max(self.n - num_samples, 0)
        return destinations

Here we create a node, put 5 cars in it and simulate 5 time steps. Note that no
more destinations are output after all 5 cars in this node has been served.

In [248]:
n1 = Node(2, 5, {'a': 0.2, 'b': 0.3, 'c': 0.5})
[n1.step() for _ in range(5)]

[['c', 'c'], ['c', 'b'], ['b'], [], []]

In [340]:
def full_network(k, lam, n):
    '''Creates a network of k nodes (fully connected graph) with equal routing
    probabilities and service times, ignoring travel times. Each node starts
    with n vehicles.'''
    prob = 1.0 / (k-1) # Equal probability to go to each node
    network = {}
    for i in range(k):
        network[i] = Node(lam, n, {j: prob for j in range(k) if j != i})
    return network

def linear_network(k, phi, lam, n):
    '''A network of nodes, with a linear virtual passenger chain from node i
    to node i+1, with service rate phi.'''
    nw = full_network(k, lam, n=n)
    for i, node in nw.items():
        if i == k-1: # Don't alter last node in chain
            continue
        node.lam = lambda x: phi + lam
        for j in node.r:
            newr = float(lam * node.r[j]) / (lam + phi)
            if j == i+1:
                newr += float(phi) / (lam + phi)
            node.r[j] = newr
        node.check_r()
    return nw            

In [341]:
test_nw = linear_network(10, 0.1, 2, 15)
print test_nw[1].r
print test_nw[9].r


{0: 0.10582010582010581, 2: 0.15343915343915343, 3: 0.10582010582010581, 4: 0.10582010582010581, 5: 0.10582010582010581, 6: 0.10582010582010581, 7: 0.10582010582010581, 8: 0.10582010582010581, 9: 0.10582010582010581}
{0: 0.1111111111111111, 1: 0.1111111111111111, 2: 0.1111111111111111, 3: 0.1111111111111111, 4: 0.1111111111111111, 5: 0.1111111111111111, 6: 0.1111111111111111, 7: 0.1111111111111111, 8: 0.1111111111111111}


In [366]:
from collections import Counter

def network_tick(network):
    '''Simulates one tick of a network'''
    dest = Counter()
    for node in network.values():
        dest.update(node.step())
    for i, count in dest.items():
        network[i].add(count)

def get_counts(network):
    return zip(*[(i, node.n) for i, node in network.items()])

test_nw = linear_network(10, 0.1, 2, 200)
counts = np.zeros(10)
for _ in range(10000):
    network_tick(test_nw)
    counts += np.array(get_counts(test_nw)[1])
print counts/10000


[  46.4689  118.3452  143.7224  164.7415  275.4051  140.2619  255.2805
  170.5065   85.0351  600.2329]
